<a href="https://colab.research.google.com/github/Beaeze/RC-AIStudy/blob/main/Practice_ANN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch  #파이토치 부르기
import pandas as pd  #Pandas : 표형태의 데이터를 다루기 위한
import numpy as np  #과학 및 수치 계산(다차원 배열)
import random  #난수 생성
import os  #os와 상호작용 기능
import scipy.special  #수학적 특수 함수 제공

## Load Data & Pre-Processing

In [1]:
import kagglehub  #케글에서 데이터 가져오기

# Download latest version
path = kagglehub.dataset_download("yashpaloswal/ann-car-sales-price-prediction")

print("Path to dataset files:", path)

100%|██████████| 29.0k/29.0k [00:00<00:00, 44.0MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/yashpaloswal/ann-car-sales-price-prediction/versions/1


In [2]:
!cp -r /root/.cache/kagglehub/datasets/yashpaloswal/ann-car-sales-price-prediction/versions/1 /content/kaggle_car_sales_data

In [14]:
data = pd.read_csv('kaggle_car_sales_data/car_purchasing.csv', encoding='latin-1')

data.head()

,customer name,customer e-mail,country,gender,age,annual Salary,credit card debt,net worth,car purchase amount
0,Martina Avila,cubilia.Curae.Phasellus@quisaccumsanconvallis.edu,Bulgaria,0,41.851720,62812.09301,11609.380910,238961.2505,35321.45877
1,Harlan Barnes,eu.dolor@diam.co.uk,Belize,0,40.870623,66646.89292,9572.957136,530973.9078,45115.52566
2,Naomi Rodriquez,vulputate.mauris.sagittis@ametconsectetueradip...,Algeria,1,43.152897,53798.55112,11160.355060,638467.1773,42925.70921
3,Jade Cunningham,malesuada@dignissim.com,Cook Islands,1,58.271369,79370.03798,14426.164850,548599.0524,67422.36313
4,Cedric Leach,felis.ullamcorper.viverra@egetmollislectus.net,Brazil,1,57.313749,59729.15130,5358.712177,560304.0671,55915.46248


In [16]:
# 1. x,y 분리 : 예측하기 위해 쓸 데이터(x), 예측할 데이터(y)

data_sp=data.drop(columns =['customer name', 'customer e-mail','country'])
data_sp.head()

,gender,age,annual Salary,credit card debt,net worth,car purchase amount
0,0,41.851720,62812.09301,11609.380910,238961.2505,35321.45877
1,0,40.870623,66646.89292,9572.957136,530973.9078,45115.52566
2,1,43.152897,53798.55112,11160.355060,638467.1773,42925.70921
3,1,58.271369,79370.03798,14426.164850,548599.0524,67422.36313
4,1,57.313749,59729.15130,5358.712177,560304.0671,55915.46248


In [17]:
# 2. train데이터와 test데이터 분리를 위한 라이브러리 불러오기
from sklearn.model_selection import train_test_split

In [22]:
# 2. train데이터와 test데이터 분리

# 80%를 학습용, 20%를 테스트용으로 나눕니다.
x_train, x_test, y_train, y_test = train_test_split(
    data_sp.drop('car purchase amount', axis=1),                     # 입력 특성 데이터 전체
    data_sp['car purchase amount'],   # 타겟/정답 레이블 데이터 전체
    test_size=0.2,         # 테스트 데이터 비율 (예: 20%)
    random_state=42,       # 결과 고정을 위한 난수 시드
    shuffle=True           # 분리 전 데이터 섞기 여부
)

print(f"data_sp의 총 행 수: {len(data_sp)}")
# 학습 데이터 크기: 입력 특성 x_train의 크기를 출력합니다.
print(f"학습 데이터 크기: {x_train.shape}")
# 테스트 데이터 크기: 입력 특성 x_test의 크기를 출력합니다.
print(f"테스트 데이터 크기: {x_test.shape}")

data_sp의 총 행 수: 500
학습 데이터 크기: (400, 5)
테스트 데이터 크기: (100, 5)


In [ ]:
x_train #-----------여기부터 하기

In [ ]:
y_train = train['support_needs']
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_train_onehot = torch.nn.functional.one_hot(y_train_tensor, num_classes=3)
print(y_train_onehot)

In [ ]:
x_train.head()

NameError: name 'x_train' is not defined

In [ ]:
categorical_cols = ['gender', 'subscription_type']
numerical_cols = [col for col in x_train.columns if col not in categorical_cols]


# pandas DataFrame -> torch.Tensor 변환
x_train_tensor = torch.tensor(x_train[numerical_cols].values, dtype=torch.float32)
x_test_tensor  = torch.tensor(x_test[numerical_cols].values, dtype=torch.float32)

# 평균, 표준편차 계산 (학습 데이터 기준)
mean = x_train_tensor.mean(dim=0, keepdim=True)
std = x_train_tensor.std(dim=0, keepdim=True)

# 표준화
x_train_tensor = (x_train_tensor - mean) / std
x_test_tensor  = (x_test_tensor - mean) / std


## Train

In [ ]:
#neural network class definition
class neuralNetwork:

    #initialize the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #가중치 행렬 wih(input_hidden)와 who(hidden_output)
        #배열 내 가중치는 w_i_j로 표기. 노드 i에서 다음 계층의 노드 j로 연결됨을 의미
        #w11 w21
        #w12 w22

        self.wih = np.random.normal(0.0, pow(self.hnodes, - 0.5),(self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, - 0.5),(self.onodes, self.hnodes))

        #learning rate
        self.lr = learningrate

        #expit()는 시그모이드 함수
        self.activation_function = lambda x: scipy.special.expit(x)

        pass

    #신경망 학습시키기
    def train(self, inputs_list, targets_list):
      #입력 리스트를 2차원의 행렬로 변환
      inputs = np.array(inputs_list, ndmin=2).T
      targets = np.array(targets_list, ndmin=2).T
# 순전파
      #은닉 계층으로 들어오는 신호를 계산
      hidden_inputs = np.dot(self.wih, inputs)
      #은닉 계층에서 나가는 신호를 계산
      hidden_outputs = self.activation_function(hidden_inputs)

      #최종 출력 계층으로 들어오는 신호를 계산
      final_inputs = np.dot(self.who, hidden_outputs)
      #최종 출력 계층에서 나가는 신호를 계산
      final_outputs = self.activation_function(final_inputs)

# 역전파
      #출력 계층의 오차는 (실제 값 - 계산 값)
      output_errors = targets - final_outputs
      #은닉 계층의 오차는 가중치에 의해 나뉜 계층의 오차들을 재조합해 계산
      hidden_errors = np.dot(self.who.T, output_errors)
# 가중치 업데이트
      #은닉 계층과 출력 계층 간의 가중치 업데이트
      self.who += self.lr*np.dot((output_errors*final_outputs*(1.0-final_outputs)), np.transpose(hidden_outputs))

      #입력 계층과 은닉 계층 간의 가중치 업데이트
      self.wih += self.lr*np.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)), np.transpose(inputs))
      pass


    #query the neural network
    def query(self, inputs_list):
      #입력 리스트를 2차원 행렬로 변환
      inputs = np.array(inputs_list, ndmin=2).T
      #은닉 계층으로 들어오는 신호를 계산
      #x_hidden = w_hidden x I (x는 가중치 행렬, I는 입력 값들의 행렬)
      #입력 계층과 은닉 계층 사이의 가중치 행렬은 입력 핼렬과 조합됨
      hidden_inputs = np.dot(self.wih, inputs)
      #은닉 계층에서 나가는 신호를 계산
      hidden_outputs = self.activation_function(hidden_inputs)
      #최종 출력 계층으로 들어오는 신호 계산
      final_inputs = np.dot(self.who, hidden_outputs)
      #최종 출력 계층에서 나가는 신호 계산
      final_outputs = self.activation_function(final_inputs)

      return final_outputs

In [ ]:
#입력, 은닉, 출력 노드의 수
input_nodes = 8
hidden_nodes = 10
output_nodes = 3

#학습률은 0.3
learning_rate = 0.1

#인경망의 인스턴스(instance)를 생성
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [ ]:
epochs = 10
for e in range(epochs):
    for i in range(len(x_train_tensor)):
        # torch.Tensor -> numpy 변환
        inputs = x_train_tensor[i].numpy()
        targets = y_train_onehot[i].numpy()   # y_train도 torch.Tensor라면 변환 필요
        n.train(inputs, targets)

    if e % 10 == 0:
        print(f"epoch: {e}")

ValueError: shapes (10,8) and (6,1) not aligned: 8 (dim 1) != 6 (dim 0)

## Prediction

In [ ]:
submission_csv = pd.read_csv('sample_submission.csv')
submission=submission_csv['support_needs']
print(submission[0])

0


In [ ]:
import numpy as np
import pandas as pd

preds = []
for i in range(len(x_test)):
    inputs = x_test.iloc[i].values
    outputs = n.query(inputs)           # shape (3,1)
    label = np.argmax(outputs)          # 가장 큰 값의 index = 클래스 예측
    preds.append(label)

# DataFrame에 예측값 저장
submission = pd.read_csv('sample_submission.csv')
submission['support_needs'] = preds
submission.to_csv('baseline_submit.csv', index=False)

print("저장 완료: baseline_submit.csv")


저장 완료: baseline_submit.csv


## Submission